# Data Loading and Database Connection for ML Forecasting

This notebook focuses on forecasting daily coffee shop sales by combining
historical sales data with external weather information stored in a
Dockerized MySQL database.

In [1]:
import os
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv

load_dotenv()

DB_NAME = os.getenv("MYSQL_DATABASE")
DB_USER = os.getenv("MYSQL_USER")
DB_PASSWORD = os.getenv("MYSQL_PASSWORD")
DB_HOST = "127.0.0.1"
DB_PORT = 3306


engine = create_engine(
    f"mysql+pymysql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
)


##### Loading Weather and Sales Data from Dockerized MySQL Database


In [2]:
coffee_sales = pd.read_sql("SELECT * FROM coffee_sales", engine)
weather_data = pd.read_sql("SELECT * FROM weather", engine)

In [ ]:
coffee_sales.head()

,transaction_id,transaction_date,transaction_time,transaction_qty,store_id,store_location,product_id,unit_price,product_category,product_type,product_detail,total_sales,month,hour,weekday,year_num,month_num,day_num
149111,149452,2023-06-30,0 days 20:18:41,2,8,Hell's Kitchen,44,2.50,Tea,Brewed herbal tea,Peppermint Rg,5.00,2023-06,20,Friday,2023,6,30
149112,149453,2023-06-30,0 days 20:25:10,2,8,Hell's Kitchen,49,3.00,Tea,Brewed Black tea,English Breakfast Lg,6.00,2023-06,20,Friday,2023,6,30
149113,149454,2023-06-30,0 days 20:31:34,1,8,Hell's Kitchen,45,3.00,Tea,Brewed herbal tea,Peppermint Lg,3.00,2023-06,20,Friday,2023,6,30
149114,149455,2023-06-30,0 days 20:57:19,1,8,Hell's Kitchen,40,3.75,Coffee,Barista Espresso,Cappuccino,3.75,2023-06,20,Friday,2023,6,30
149115,149456,2023-06-30,0 days 20:57:19,2,8,Hell's Kitchen,64,0.80,Flavours,Regular syrup,Hazelnut syrup,1.60,2023-06,20,Friday,2023,6,30


In [5]:
weather_data.head()

,date,temperature_2m_mean
0,2023-01-01,8.78542
1,2023-01-02,7.98125
2,2023-01-03,9.69167
3,2023-01-04,13.08130
4,2023-01-05,9.91250


In [ ]:
## coffee_sales.drop(columns=["transaction_time"], inplace=True)


In [6]:
sales_ml = coffee_sales[
    [
        "day_num",
        "month_num",
        "year_num",
        "store_id",
        "total_sales",
        "transaction_date"
    ]
]

In [8]:
daily_sales = sales_ml.groupby(
    ["transaction_date", "store_id"]
).agg({
    "total_sales": "sum",
    "year_num": "first",
    "month_num": "first",
    "day_num": "first"
}).reset_index()

daily_sales

,transaction_date,store_id,total_sales,year_num,month_num,day_num
0,2023-01-01,3,868.40,2023,1,1
1,2023-01-01,5,788.35,2023,1,1
2,2023-01-01,8,851.45,2023,1,1
3,2023-01-02,3,925.50,2023,1,2
4,2023-01-02,5,649.05,2023,1,2
...,...,...,...,...,...,...
538,2023-06-29,5,1299.45,2023,6,29
539,2023-06-29,8,1298.55,2023,6,29
540,2023-06-30,3,1807.65,2023,6,30
541,2023-06-30,5,1768.74,2023,6,30


In [9]:
daily_sales.rename(
    columns={"transaction_date": "date"},
    inplace=True
)

In [11]:
weather_data["date"] = pd.to_datetime(weather_data["date"])
daily_sales["date"] = pd.to_datetime(daily_sales["date"])

In [13]:
ml_df = pd.merge(
    daily_sales,
    weather_data,
    on="date",
    how="left"
)

In [14]:
ml_df.drop(columns=["date"], inplace=True)

In [15]:
ml_df

,store_id,total_sales,year_num,month_num,day_num,temperature_2m_mean
0,3,868.40,2023,1,1,8.78542
1,5,788.35,2023,1,1,8.78542
2,8,851.45,2023,1,1,8.78542
3,3,925.50,2023,1,2,7.98125
4,5,649.05,2023,1,2,7.98125
...,...,...,...,...,...,...
538,5,1299.45,2023,6,29,22.35420
539,8,1298.55,2023,6,29,22.35420
540,3,1807.65,2023,6,30,23.03960
541,5,1768.74,2023,6,30,23.03960
